# Week 1: Introduction to Networks

**Learning objectives** — After this lab you should be able to:

- Explain what nodes and edges represent in a network
- Build a small graph from scratch with NetworkX
- Distinguish directed, undirected, and weighted graphs
- Load real-world datasets and inspect basic properties
- Visualize networks with different layouts and coloring

Networks are everywhere — social media friendships, airline routes, protein interactions, the web itself.
In this first lab we will learn the vocabulary of **graph theory** and get hands-on with NetworkX,
the Python library we will use throughout the course.

## 0. Environment Smoke Test

Run the cell below to verify your environment is set up correctly.

In [ ]:
# === Environment Smoke Test ===
# Run this cell first. If it passes, your environment is correctly set up.
# If it fails, follow the error message instructions.


def _smoke_test():
    """Validate that all required packages are installed and importable."""
    import sys

    errors = []

    # Check Python version
    if sys.version_info < (3, 12):
        errors.append(
            f"Python >= 3.12 required, but you have {sys.version}.\n"
            f"  Fix: run 'uv sync' from the repository root to set up the correct environment."
        )

    # Check required packages and minimum versions
    checks = [
        ("networkx", "nx", "3.6"),
        ("matplotlib", "matplotlib", "3.10"),
        ("numpy", "np", "2.2"),
        ("scipy", "scipy", "1.14"),
        ("pandas", "pd", "2.2"),
        ("seaborn", "sns", "0.13"),
        ("pyvis", "pyvis", "0.3"),
        ("ipywidgets", "ipywidgets", "8.1"),
    ]

    for package_name, import_name, min_version in checks:
        try:
            mod = __import__(package_name)
            version = getattr(mod, "__version__", "0.0.0")
            # Compare major.minor only
            installed = tuple(int(x) for x in version.split(".")[:2])
            required = tuple(int(x) for x in min_version.split(".")[:2])
            if installed < required:
                errors.append(
                    f"{package_name} >= {min_version} required, but {version} is installed.\n"
                    f"  Fix: run 'uv sync' from the repository root."
                )
        except ImportError:
            errors.append(
                f"{package_name} is not installed.\n"
                f"  Fix: run 'uv sync' from the repository root."
            )

    # Check netsci package
    try:
        import netsci
    except ImportError:
        errors.append(
            "The 'netsci' package is not installed.\n"
            "  Fix: run 'uv sync' from the repository root.\n"
            "  This installs the course's shared utilities package."
        )

    if errors:
        print("ENVIRONMENT CHECK FAILED")
        print("=" * 50)
        for i, error in enumerate(errors, 1):
            print(f"\n  {i}. {error}")
        print("\n" + "=" * 50)
        print("Run 'uv sync' in your terminal, then restart the kernel and try again.")
        raise SystemExit(1)
    else:
        import networkx as nx

        print("Environment check passed!")
        print(f"  Python:    {sys.version.split()[0]}")
        print(f"  NetworkX:  {nx.__version__}")
        print(f"  netsci:    {netsci.__version__}")
        print("You're ready to start the lab.")


_smoke_test()
del _smoke_test  # clean up namespace

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from netsci.loaders import load_graph
from netsci.utils import SEED, graph_summary
from netsci import viz

---
## 1. What is a Graph?

Think of a social network: **people** are *nodes* (also called *vertices*) and **friendships** are *edges* (also called *links*).
That's all a graph is — a collection of nodes connected by edges.

Let's build one from scratch.

In [ ]:
# Create an empty graph and add 5 people
G = nx.Graph()
G.add_nodes_from(["Alice", "Bob", "Carol", "Dave", "Eve"])

# Add friendships (edges)
G.add_edge("Alice", "Bob")
G.add_edge("Alice", "Carol")
G.add_edge("Bob", "Carol")
G.add_edge("Bob", "Dave")
G.add_edge("Dave", "Eve")

print("Nodes:", list(G.nodes()))
print("Edges:", list(G.edges()))
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

In [ ]:
viz.draw_graph(G, title="Our first graph")

In [ ]:
# Annotated graph anatomy — labeling key concepts
with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(6, 5))
    pos = nx.spring_layout(G, seed=SEED)
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color="#cccccc", width=1.5)
    nx.draw_networkx_nodes(G, pos, ax=ax, node_color="#4878CF", node_size=300)
    nx.draw_networkx_labels(G, pos, ax=ax, font_size=8, font_color="white")

    # Annotate "node"
    alice_pos = pos["Alice"]
    ax.annotate(
        "node",
        xy=alice_pos,
        xytext=(alice_pos[0] - 0.25, alice_pos[1] + 0.25),
        fontsize=11,
        fontweight="bold",
        color="#333333",
        arrowprops=dict(arrowstyle="->", color="#333333", lw=1.5),
    )

    # Annotate "edge" with more spacing to avoid overlap
    bob_pos, carol_pos = pos["Bob"], pos["Carol"]
    mid = ((bob_pos[0] + carol_pos[0]) / 2, (bob_pos[1] + carol_pos[1]) / 2)
    ax.annotate(
        "edge",
        xy=mid,
        xytext=(mid[0] + 0.4, mid[1] + 0.3),  # bumped further right and up
        fontsize=11,
        fontweight="bold",
        color="#333333",
        arrowprops=dict(arrowstyle="->", color="#333333", lw=1.5),
    )

    # Annotate degree with adjusted offset
    bob_deg = G.degree("Bob")
    ax.annotate(
        f"degree = {bob_deg}",
        xy=pos["Bob"],
        xytext=(pos["Bob"][0] + 0.4, pos["Bob"][1] - 0.3),  # moved a bit farther down/right
        fontsize=11,
        fontweight="bold",
        color="#333333",
        arrowprops=dict(arrowstyle="->", color="#333333", lw=1.5),
    )

    ax.set_title("Graph Anatomy: Nodes, Edges, and Degree", fontsize=13)
    ax.axis("off")
    fig.tight_layout()
    plt.show()

Each circle is a node, each line is an edge.  Notice that Alice, Bob, and Carol form a **triangle** — they are all friends with each other.

---
## 2. Directed vs. Undirected Graphs

On **Facebook**, friendship goes both ways — if Alice is friends with Bob, Bob is also friends with Alice. That's an *undirected* graph.

On **Twitter/X**, you can follow someone without them following you back. That's a *directed* graph — edges have arrows.

In [ ]:
# Create a directed graph ("follows" relationships)
D = nx.DiGraph()
D.add_edge("Alice", "Bob")  # Alice follows Bob
D.add_edge("Bob", "Alice")  # Bob follows Alice back
D.add_edge("Carol", "Alice")  # Carol follows Alice (not mutual)
D.add_edge("Carol", "Bob")
D.add_edge("Dave", "Alice")

print("Is directed?", D.is_directed())
print()

# In directed graphs, in-degree != out-degree
for node in D.nodes():
    print(f"{node}: in-degree={D.in_degree(node)}, out-degree={D.out_degree(node)}")

In [ ]:
viz.draw_graph(D, title="Directed graph (Twitter-style follows)")

Alice has a high **in-degree** (many followers) but a low **out-degree** (she follows few people). In directed networks, popularity and activity are different things.

### Real-World Directed Network: Email Communication

The toy example above illustrates the concept, but directed networks are most interesting at scale.
Let's load the EU email dataset — 1,005 employees, 25,571 emails — and explore how in-degree and out-degree tell different stories.

In [ ]:
# Load the email network as a directed graph
G_email = load_graph("email")
graph_summary(G_email)

print(f"\nIs directed? {G_email.is_directed()}")
print(f"Nodes: {G_email.number_of_nodes()}, Edges: {G_email.number_of_edges()}")

In [ ]:
# In-degree vs out-degree distributions
in_degrees = [d for _, d in G_email.in_degree()]
out_degrees = [d for _, d in G_email.out_degree()]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(in_degrees, bins=50, edgecolor="white", alpha=0.8, color="#4878CF")
axes[0].set_xlabel("In-degree")
axes[0].set_ylabel("Count")
axes[0].set_title("Email — In-degree Distribution\n(How many emails received)")

axes[1].hist(out_degrees, bins=50, edgecolor="white", alpha=0.8, color="#D65F5F")
axes[1].set_xlabel("Out-degree")
axes[1].set_ylabel("Count")
axes[1].set_title("Email — Out-degree Distribution\n(How many emails sent)")

fig.tight_layout()
plt.show()

In [ ]:
# Nodes with high in-degree but low out-degree (and vice versa)
nodes = list(G_email.nodes())
in_deg = dict(G_email.in_degree())
out_deg = dict(G_email.out_degree())

# High in, low out — "popular recipients" (managers? mailing lists?)
high_in_low_out = [
    (n, in_deg[n], out_deg[n]) for n in nodes if in_deg[n] > 50 and out_deg[n] < 20
]
high_in_low_out.sort(key=lambda x: x[1], reverse=True)
print("High in-degree, low out-degree (popular recipients):")
for n, ind, outd in high_in_low_out[:5]:
    print(f"  Node {n}: in={ind}, out={outd}")

# High out, low in — "prolific senders"
high_out_low_in = [
    (n, in_deg[n], out_deg[n]) for n in nodes if out_deg[n] > 50 and in_deg[n] < 20
]
high_out_low_in.sort(key=lambda x: x[2], reverse=True)
print(f"\nHigh out-degree, low in-degree (prolific senders):")
for n, ind, outd in high_out_low_in[:5]:
    print(f"  Node {n}: in={ind}, out={outd}")

### Weakly vs Strongly Connected Components

In directed graphs, connectivity has two flavors:
- **Weakly connected**: there is a path between every pair of nodes *if we ignore edge directions*
- **Strongly connected**: there is a directed path from every node to every other node (following arrows)

A strongly connected component is a much stricter requirement — it means information can flow in *both* directions between every pair.

In [ ]:
# Weakly vs strongly connected components
n_weak = nx.number_weakly_connected_components(G_email)
n_strong = nx.number_strongly_connected_components(G_email)
largest_scc = max(nx.strongly_connected_components(G_email), key=len)

print(f"Weakly connected components:  {n_weak}")
print(f"Strongly connected components: {n_strong}")
print(
    f"Largest SCC: {len(largest_scc)} nodes ({len(largest_scc) / G_email.number_of_nodes():.0%} of network)"
)
print(f"\nThe gap tells us: most nodes can be reached (weakly), but")
print(f"reciprocal communication paths are rarer (strongly).")

---
## 2.5 Bipartite Networks (a brief look)

Some networks naturally have **two types of nodes**. Movies and actors. Papers and authors. Customers and products. These are **bipartite** (or "two-mode") networks — edges only connect nodes of *different* types.

Let's build a tiny example.

In [ ]:
# A tiny bipartite graph: movies and actors
B = nx.Graph()
# Add nodes with bipartite attribute
movies = ["Inception", "The Matrix", "Interstellar"]
actors = ["DiCaprio", "Reeves", "Hathaway", "Fishburne"]
B.add_nodes_from(movies, bipartite=0)
B.add_nodes_from(actors, bipartite=1)

# Edges connect actors to movies they appeared in
B.add_edges_from(
    [
        ("DiCaprio", "Inception"),
        ("Hathaway", "Interstellar"),
        ("Hathaway", "The Matrix"),
        ("Reeves", "The Matrix"),
        ("Fishburne", "The Matrix"),
    ]
)

# Create two nicely separated columns and center them to avoid overlapping y positions
n_movies = len(movies)
n_actors = len(actors)
gap = 1.6
# Center each column around y=0 and add a small vertical offset for the actors column
y_movies = np.linspace((n_movies - 1) * gap / 2, - (n_movies - 1) * gap / 2, n_movies)
y_actors = np.linspace((n_actors - 1) * gap / 2 + 0.35, - (n_actors - 1) * gap / 2 + 0.35, n_actors)

pos = {}
for i, m in enumerate(movies):
    pos[m] = (0.0, y_movies[i])
for i, a in enumerate(actors):
    pos[a] = (1.0, y_actors[i])

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(9, 5))
    nx.draw_networkx_nodes(
        B,
        pos,
        nodelist=movies,
        node_color="#4878CF",
        node_shape="s",
        node_size=700,
        ax=ax,
        label="Movies",
        linewidths=0.8,
        edgecolors="#2f4f77",
    )
    nx.draw_networkx_nodes(
        B,
        pos,
        nodelist=actors,
        node_color="#D65F5F",
        node_size=550,
        ax=ax,
        label="Actors",
        linewidths=0.8,
        edgecolors="#7a2b2b",
    )
    nx.draw_networkx_edges(B, pos, ax=ax, edge_color="#999999", width=1.5, alpha=0.9)

    # Draw labels offset from nodes with a white background box for readability; increase horizontal offset
    for n, (x, y) in pos.items():
        is_movie = B.nodes[n].get("bipartite", 0) == 0
        if is_movie:
            ax.text(
                x - 0.18,
                y,
                n,
                fontsize=11,
                ha="right",
                va="center",
                bbox=dict(facecolor="white", edgecolor="none", pad=0.35, alpha=0.95),
                zorder=3,
            )
        else:
            ax.text(
                x + 0.18,
                y,
                n,
                fontsize=11,
                ha="left",
                va="center",
                bbox=dict(facecolor="white", edgecolor="none", pad=0.35, alpha=0.95),
                zorder=3,
            )

    ax.legend(scatterpoints=1, fontsize=9)
    ax.set_xlim(-0.9, 1.9)
    ax.set_title("Bipartite Network: Movies and Actors")
    ax.axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
# One-mode projection: which actors co-starred together?
actor_set = {n for n, d in B.nodes(data=True) if d["bipartite"] == 1}
G_actors = nx.bipartite.projected_graph(B, actor_set)

print("Actor co-starring network (projected from bipartite):")
for u, v in G_actors.edges():
    shared = set(B.neighbors(u)) & set(B.neighbors(v))
    print(f"  {u} -- {v}  (shared movies: {shared})")

viz.draw_graph(G_actors, title="Actor Projection — who co-starred?")

**Many real networks are secretly bipartite.** Co-authorship networks (papers × authors), product recommendation networks (users × items), and even board-of-directors networks (companies × people) all start as two-mode graphs. The "one-mode projection" collapses them into a single-type network — but information is lost in the process. Keep this in mind when you encounter a dense network: it may be the projection of a simpler bipartite structure.

**Looking ahead**: The projection above creates implicit communities — actors who share many movies form tight clusters. In Week 5 we will learn algorithms that detect such communities automatically. And the dense triangles that projections create inflate the clustering coefficient (Week 2) — a useful reminder that network properties depend on how the data was constructed.

---
## 3. Weighted Graphs

Sometimes edges carry extra information. In a co-appearance network, the **weight** of an edge tells us how many times two characters appeared together.

**Dataset**: The *Les Miserables* co-appearance network was compiled by Donald Knuth (1993) from Victor Hugo's novel. Each **node** is a character, each **edge** means two characters appeared in the same chapter, and the **weight** counts how many chapters they co-appeared in. It is a classic benchmark because it has clear community structure (the subplots of the novel) and a dominant hub (Jean Valjean).

In [ ]:
# Build a small weighted graph
W = nx.Graph()
W.add_edge("Alice", "Bob", weight=5)  # Met 5 times
W.add_edge("Alice", "Carol", weight=2)  # Met 2 times
W.add_edge("Bob", "Carol", weight=8)  # Met 8 times

for u, v, data in W.edges(data=True):
    print(f"{u} -- {v}  weight={data['weight']}")

In [ ]:
# Les Miserables co-appearance network — a classic weighted graph
G_les = load_graph("lesmis")
graph_summary(G_les)

In [ ]:
# Show a few edges with their weights
edges_with_weight = [(u, v, d["weight"]) for u, v, d in G_les.edges(data=True)]
edges_with_weight.sort(key=lambda x: x[2], reverse=True)
print("Top 5 strongest connections:")
for u, v, w in edges_with_weight[:5]:
    print(f"  {u} -- {v}: weight = {w}")

In [ ]:
# Draw Les Mis — node size proportional to degree
degrees = dict(G_les.degree())
node_sizes = [degrees[n] * 20 for n in G_les.nodes()]
viz.draw_graph(G_les, node_size=node_sizes, title="Les Miserables co-appearances")

In [ ]:
# Weighted edge visualization — thicker edges = stronger co-appearance
weights = [G_les[u][v]["weight"] for u, v in G_les.edges()]
max_w = max(weights)
norm_widths = [3.0 * w / max_w for w in weights]

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(9, 7))
    pos = nx.spring_layout(G_les, seed=SEED, k=0.3)
    degrees = dict(G_les.degree())
    node_sizes = [degrees[n] * 20 for n in G_les.nodes()]
    nx.draw_networkx_edges(
        G_les, pos, ax=ax, width=norm_widths, edge_color="#999999", alpha=0.6
    )
    nx.draw_networkx_nodes(
        G_les, pos, ax=ax, node_size=node_sizes, node_color="#4878CF", alpha=0.85
    )
    # Label only high-degree nodes
    top_nodes = sorted(degrees, key=degrees.get, reverse=True)[:8]
    labels = {n: n for n in top_nodes}
    nx.draw_networkx_labels(G_les, pos, labels, ax=ax, font_size=8)
    ax.set_title("Les Miserables — edge width ∝ co-appearance weight", fontsize=13)
    ax.axis("off")
    fig.tight_layout()
    plt.show()

**Edge weights encode relationship strength.** Thicker lines connect characters who appear together more often. Notice how central characters form a dense weighted core — Valjean and Cosette share the strongest connection (weight 31), followed by Cosette–Marius and Valjean–Marius. The revolutionary group (Enjolras, Courfeyrac) and the Valjean–Javert rivalry also form dense weighted clusters, reflecting the plot's central relationships.

**What to notice**: Valjean dominates the layout — his node is the largest because he co-appears with more characters than anyone else. He is the network's **hub**. The smaller clusters around him correspond to different subplots of the novel (the Thénardiers, the students at the barricade, the Bishop's household). We will learn to detect these clusters automatically in Week 5.

---
## 4. Real-World Network: Zachary's Karate Club

In the 1970s, sociologist Wayne Zachary studied a university karate club for two years.
During his study, a conflict arose between the instructor (node 0) and the club president (node 33),
and the club eventually split into two factions.

This is one of the most famous datasets in network science — let's explore it.

In [ ]:
G_karate = load_graph("karate")
graph_summary(G_karate)

In [ ]:
viz.draw_graph(G_karate, title="Zachary's Karate Club")

In [ ]:
# Color nodes by the faction they joined after the split
# The 'club' attribute is 'Mr. Hi' (instructor) or 'Officer' (president)
color_map = []
for node in G_karate.nodes():
    club = G_karate.nodes[node].get("club", "Mr. Hi")
    color_map.append("#D65F5F" if club == "Mr. Hi" else "#4878CF")

viz.draw_graph(G_karate, node_color=color_map, title="Karate Club — colored by faction")

**Interpreting the factions**: Node 0 (the instructor) and node 33 (the president) are the two highest-degree nodes — they serve as the "centers" of their respective factions. Most members connect primarily within their own faction, but a few **bridge nodes** (like nodes 2 and 8) have connections to both sides. These bridges are exactly the members whose allegiance was hardest to predict during the real split.

In [ ]:
viz.plot_adjacency(G_karate, title="Karate Club — Adjacency Matrix")

Notice the block structure in the adjacency matrix — members within the same faction are more densely connected. This foreshadows the **community detection** topic in Week 5.

---
## 5. Exploring with NetworkX

NetworkX provides many functions to query a graph. Let's try a few essential ones.

In [ ]:
# Degree: how many connections each node has
print("Degree of node 0 (instructor):", G_karate.degree(0))
print("Degree of node 33 (president):", G_karate.degree(33))
print()

# Neighbors: who is node 0 connected to?
print("Neighbors of node 0:", list(G_karate.neighbors(0)))

In [ ]:
# Shortest path between two nodes
path = nx.shortest_path(G_karate, source=0, target=33)
print(f"Shortest path from node 0 to node 33: {path}")
print(f"Path length: {len(path) - 1} edges")

**Try it yourself**: In the cell below, try pairs from opposite factions (e.g., source=5, target=30 or source=1, target=32). Do cross-faction paths tend to be longer than within-faction paths? What is the shortest a cross-faction path can be?

In [ ]:
# ---- TWEAK: Pick two different nodes and find the shortest path ----
source_node = 5  # <-- change me
target_node = 30  # <-- change me

path = nx.shortest_path(G_karate, source=source_node, target=target_node)
print(f"Shortest path from {source_node} to {target_node}: {path}")
print(f"Path length: {len(path) - 1} edges")

**What you should see**: Within-faction pairs (e.g., 0→1, 33→31) typically have paths of length 1-2. Cross-faction pairs often require 2-3 hops — they must pass through one of the bridge nodes or through the two leaders. The diameter of this small network is only 5, so even the most distant pair is surprisingly close.

---
## 6. Tweak & Observe: Layouts and Degree Distribution

The way we draw a graph is a *choice* — the same graph can look very different with different layout algorithms.

**Predict before you run**: Do you think different layout algorithms will preserve the two-faction structure equally well? A **spring layout** simulates physical forces (connected nodes attract), **Kamada-Kawai** minimizes stress in distances, and **circular** simply arranges nodes in a ring. Which one do you expect to show the factions most clearly?

In [ ]:
# ---- TWEAK: Try different layouts ----
layout = "spring"  # <-- change me: "spring", "kamada_kawai", or "circular"

viz.draw_graph(
    G_karate,
    node_color=color_map,
    layout=layout,
    title=f"Karate Club — {layout} layout",
)

In [ ]:
# Compare all three layouts side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

layouts = {
    "spring": nx.spring_layout(G_karate, seed=SEED),
    "kamada_kawai": nx.kamada_kawai_layout(G_karate),
    "circular": nx.circular_layout(G_karate),
}

for ax, (name, pos) in zip(axes, layouts.items()):
    nx.draw_networkx(
        G_karate,
        pos,
        ax=ax,
        node_color=color_map,
        node_size=80,
        edge_color="#cccccc",
        width=0.5,
        with_labels=True,
        font_size=7,
        alpha=0.9,
    )
    ax.set_title(name)
    ax.axis("off")

fig.suptitle("Same graph, three layouts", fontsize=14)
fig.tight_layout()
plt.show()

**Layouts compared**: Spring and Kamada-Kawai both pull the two factions apart — connected nodes cluster together, revealing the community structure. The circular layout obscures this structure because it ignores connectivity. The takeaway: **layouts are visualizations, not measurements** — they can suggest structure but never prove it. For rigorous community detection, we will use algorithms in Week 5.

In [ ]:
viz.plot_degree_dist(G_karate, title="Karate Club — Degree Distribution")

In [ ]:
viz.plot_degree_dist(G_les, title="Les Miserables — Degree Distribution")

Both distributions are **right-skewed** — most nodes have few connections, but a few "stars" (Valjean in Les Mis, the instructor and president in Karate) have many. This pattern appears in almost every real-world network.

**Try it yourself**: Find the node with the highest degree in the Karate Club using a one-liner. Fill in the cell below and run it to check.

In [ ]:
# YOUR CODE HERE
highest_degree_node = max(G_karate.nodes(), key=lambda n: G_karate.degree(n))
assert highest_degree_node == 33, (
    "Hint: use max() with a key function that returns the degree of each node"
)
print(
    f"Node {highest_degree_node} has the highest degree: {G_karate.degree(highest_degree_node)}"
)

**Looking ahead**: Why do so many real networks have this skewed "few hubs, many leaves" pattern? In Week 4 we will discover that a simple mechanism — **preferential attachment** ("the rich get richer") — naturally produces these fat-tailed degree distributions.

---
## Summary

In this lab we learned:

| Concept | Key idea |
|---------|----------|
| **Node & edge** | The building blocks of every network |
| **Undirected vs directed** | Facebook (mutual) vs Twitter (one-way) |
| **In-degree vs out-degree** | Popularity vs activity in directed networks |
| **Weakly vs strongly connected** | Ignoring vs respecting edge direction |
| **Weighted edges** | Edges can carry strength or frequency |
| **Bipartite networks** | Two node types, edges only cross types |
| **Degree** | Number of connections a node has |
| **Shortest path** | Fewest hops between two nodes |
| **Degree distribution** | How degrees are spread across the network |

Next week we will dive deeper into **network properties** — centrality, clustering, and what makes some nodes more important than others.